In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
warnings.filterwarnings('ignore')

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [3]:
delimiters=["।","?","!"]

In [4]:
sg=Word2Vec.load('sg/or-d100-m2-sg.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def o2h(text):
    return transliterate(text, sanscript.ORIYA, sanscript.DEVANAGARI)

In [8]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [9]:
def word_embedding(word):
    try:
        return sg.wv[o2h(word)]
    except:
        return np.array([0.0]*100)

In [10]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [11]:
def ed(a,b):
    return lin.norm(a-b)

In [12]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [13]:
data=pd.read_csv('../odia.csv')

In [14]:
query='ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କରିବାକୁ ଯାଇ'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [15]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [16]:
verse_index=max_cosine(query_embedding)

In [17]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
589,17,3,सत्त्वानुरूपा सर्वस्य श्रद्धा भवति भारत।श्रद्ध...,ପୂର୍ବ ଶ୍ଲୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିଲା ଯେ ଆମେ କୌଣସି...
80,2,41,व्यवसायात्मिका बुद्धिरेकेह कुरुनन्दन।\n\nबहुशा...,ଆସକ୍ତି ମନର ଏକ ପ୍ରକ୍ରିୟା ଅଟେ । ମନ ଯାହା ପ୍ରତି ଆସ...
41,2,2,श्री भगवानुवाच\n\nकुतस्त्वा कश्मलमिदं विषमे सम...,ଆମ ଧର୍ମଗ୍ରନ୍ଥମାନଙ୍କରେ ଆର୍ଯ୍ୟ ଶବ୍ଦଟି କୌଣସି ଏକ ଜ...
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
363,9,33,किं पुनर्ब्राह्मणाः पुण्या भक्ता राजर्षयस्तथा।...,ଯଦି ଅତ୍ୟନ୍ତ ପାପୀମାନେ ମଧ୍ୟ ଭକ୍ତି ମାର୍ଗରେ ସଫଳତା ...
170,4,16,किं कर्म किमकर्मेति कवयोऽप्यत्र मोहिताः।\n\nतत...,ଧର୍ମର ନିୟମ ମାନସିକ ଚିନ୍ତନ ଦ୍ୱାରା ସ୍ଥିର କରାଯାଇ ପ...
72,2,33,अथ चैत्त्वमिमं धर्म्यं संग्रामं न करिष्यसि।\n\...,ଯଦି ଜଣେ ଯୋଦ୍ଧା ଯୁଦ୍ଧକ୍ଷେତ୍ରରେ ଅହିଂସା ଆଚରଣ କରନ୍...
283,7,11,बलं बलवतां चाहं कामरागविवर्जितम्।\n\nधर्माविरु...,ଅପ୍ରାପ୍ତ ବସ୍ତୁର ସକ୍ରିୟ କାମନାକୁ ବାସନା କୁହାଯାଏ ।...
402,10,38,दण्डो दमयतामस्मि नीतिरस्मि जिगीषताम्।\n\nमौनं ...,ମନୁଷ୍ୟର ପ୍ରକୃତି ଏପରି ଯେ ସେମାନଙ୍କୁ ସତ୍‌କର୍ମରେ ପ...


## Euclidean Distance

In [18]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [19]:
verse_index=max_ed(query_embedding)

In [20]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
589,17,3,सत्त्वानुरूपा सर्वस्य श्रद्धा भवति भारत।श्रद्ध...,ପୂର୍ବ ଶ୍ଲୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିଲା ଯେ ଆମେ କୌଣସି...
41,2,2,श्री भगवानुवाच\n\nकुतस्त्वा कश्मलमिदं विषमे सम...,ଆମ ଧର୍ମଗ୍ରନ୍ଥମାନଙ୍କରେ ଆର୍ଯ୍ୟ ଶବ୍ଦଟି କୌଣସି ଏକ ଜ...
80,2,41,व्यवसायात्मिका बुद्धिरेकेह कुरुनन्दन।\n\nबहुशा...,ଆସକ୍ତି ମନର ଏକ ପ୍ରକ୍ରିୟା ଅଟେ । ମନ ଯାହା ପ୍ରତି ଆସ...
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
72,2,33,अथ चैत्त्वमिमं धर्म्यं संग्रामं न करिष्यसि।\n\...,ଯଦି ଜଣେ ଯୋଦ୍ଧା ଯୁଦ୍ଧକ୍ଷେତ୍ରରେ ଅହିଂସା ଆଚରଣ କରନ୍...
170,4,16,किं कर्म किमकर्मेति कवयोऽप्यत्र मोहिताः।\n\nतत...,ଧର୍ମର ନିୟମ ମାନସିକ ଚିନ୍ତନ ଦ୍ୱାରା ସ୍ଥିର କରାଯାଇ ପ...
283,7,11,बलं बलवतां चाहं कामरागविवर्जितम्।\n\nधर्माविरु...,ଅପ୍ରାପ୍ତ ବସ୍ତୁର ସକ୍ରିୟ କାମନାକୁ ବାସନା କୁହାଯାଏ ।...
402,10,38,दण्डो दमयतामस्मि नीतिरस्मि जिगीषताम्।\n\nमौनं ...,ମନୁଷ୍ୟର ପ୍ରକୃତି ଏପରି ଯେ ସେମାନଙ୍କୁ ସତ୍‌କର୍ମରେ ପ...
164,4,10,वीतरागभयक्रोधा मन्मया मामुपाश्रिताः।\n\nबहवो ज...,"ପୂର୍ବବର୍ତ୍ତୀ ଶ୍ଲୋକରେ ଶ୍ରୀକୃଷ୍ଣ କହିଲେ ଯେ, ଯେଉଁମ..."


# Mean Pooling

## Cosine Similarity

In [21]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [22]:
verse_index=mean_cosine(query_embedding)

In [23]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
155,4,1,श्री भगवानुवाच\n\nइमं विवस्वते योगं प्रोक्तवान...,କାହାକୁ କେବଳ ଅମୂଲ୍ୟ ଜ୍ଞାନ ପ୍ରଦାନ କରିଦେଲେ ଯଥେଷ୍ଟ...
78,2,39,एषा तेऽभिहिता सांख्ये बुद्धिर्योगे त्विमां श्र...,ସାଂଖ୍ୟ ଶବ୍ଦଟି ସାଂ ଅର୍ଥାତ୍ ‘ପୂର୍ଣ୍ଣ’ ଓ ଖ୍ୟ ଅର୍ଥ...
628,18,14,अधिष्ठानं तथा कर्ता करणं च पृथग्विधम्।विविधाश्...,"ଏହି ଶ୍ଳୋକରେ, ଅଧିଷ୍ଠାନଂ ଶବ୍ଦର ଅର୍ଥ “ବାସସ୍ଥାନ” ଅ..."
250,6,25,शनैः शनैरुपरमेद् बुद्ध्या धृतिगृहीतया।\n\nआत्म...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
249,6,24,सङ्कल्पप्रभवान्कामांस्त्यक्त्वा सर्वानशेषतः।\n...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
115,3,4,न कर्मणामनारम्भान्नैष्कर्म्यं पुरुषोऽश्नुते।\n...,ଏହି ଶ୍ଲୋକର ପ୍ରଥମ ପଂକ୍ତି କର୍ମଯୋଗୀ (କର୍ମ ମାର୍ଗର ...
222,5,26,कामक्रोधवियुक्तानां यतीनां यतचेतसाम्।\n\nअभितो...,ଶ୍ଲୋକ ୫.୨ରେ ବର୍ଣ୍ଣନା କରାଯାଇଛି ଯେ କର୍ମଯୋଗ ଅଧିକା...
63,2,24,अच्छेद्योऽयमदाह्योऽयमक्लेद्योऽशोष्य एव च।\n\nन...,ଆତ୍ମାର ଅମରତ୍ୱ ବିଷୟରେ ଏଠାରେ ପୁନରାବୃତ୍ତି କରାଯାଇଛ...
170,4,16,किं कर्म किमकर्मेति कवयोऽप्यत्र मोहिताः।\n\nतत...,ଧର୍ମର ନିୟମ ମାନସିକ ଚିନ୍ତନ ଦ୍ୱାରା ସ୍ଥିର କରାଯାଇ ପ...


## Euclidean Distance

In [24]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [25]:
verse_index=mean_ed(query_embedding)

In [26]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
155,4,1,श्री भगवानुवाच\n\nइमं विवस्वते योगं प्रोक्तवान...,କାହାକୁ କେବଳ ଅମୂଲ୍ୟ ଜ୍ଞାନ ପ୍ରଦାନ କରିଦେଲେ ଯଥେଷ୍ଟ...
146,3,35,श्रेयान्स्वधर्मो विगुणः परधर्मात्स्वनुष्ठितात्...,ଏହି ଶ୍ଲୋକରେ ଧର୍ମ ଶବ୍ଦର ବ୍ୟବହାର ଚାରିଥର କରାଯାଇଛି...
62,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"ଚୈତନ୍ୟତା, ଯାହା ଆତ୍ମାର ଲକ୍ଷଣ ଅଟେ, ତାହାର ଅନୁଭବ ମ..."
222,5,26,कामक्रोधवियुक्तानां यतीनां यतचेतसाम्।\n\nअभितो...,ଶ୍ଲୋକ ୫.୨ରେ ବର୍ଣ୍ଣନା କରାଯାଇଛି ଯେ କର୍ମଯୋଗ ଅଧିକା...
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
250,6,25,शनैः शनैरुपरमेद् बुद्ध्या धृतिगृहीतया।\n\nआत्म...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
249,6,24,सङ्कल्पप्रभवान्कामांस्त्यक्त्वा सर्वानशेषतः।\n...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
633,18,19,ज्ञानं कर्म च कर्ता च त्रिधैव गुणभेदतः।प्रोच्य...,ଶ୍ରୀକୃଷ୍ଣ ପୁନର୍ବାର ପ୍ରକୃତିର ତିନିଗୁଣ ବିଷୟରେ ଆଲୋ...
627,18,13,पञ्चैतानि महाबाहो कारणानि निबोध मे।सांख्ये कृत...,"ଫଳାସକ୍ତି ରହିତ ହୋଇ କର୍ମ କରାଯାଇପାରିବ, ଏହା ଜାଣିବା..."


# Line Matching

## Cosine Similarity

In [27]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [28]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
45,2,6,न चैतद्विद्मः कतरन्नो गरीयो\n\nयद्वा जयेम यदि ...,ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କ...
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
152,3,41,तस्मात्त्वमिन्द्रियाण्यादौ नियम्य भरतर्षभ।\n\n...,"ବର୍ତ୍ତମାନ ଶ୍ରୀକୃଷ୍ଣ, ସମସ୍ତ ଅନିଷ୍ଟର ମୂଳ କାରଣ ତଥ..."
552,15,10,उत्क्रामन्तं स्थितं वापि भुञ्जानं वा गुणान्वित...,ଯଦିଓ ଆତ୍ମା ଶରୀର ମଧ୍ୟରେ ବାସ କରେ ଏବଂ ମନ ଓ ଇନ୍ଦ୍ର...
628,18,14,अधिष्ठानं तथा कर्ता करणं च पृथग्विधम्।विविधाश्...,"ଏହି ଶ୍ଳୋକରେ, ଅଧିଷ୍ଠାନଂ ଶବ୍ଦର ଅର୍ଥ “ବାସସ୍ଥାନ” ଅ..."
136,3,25,सक्ताः कर्मण्यविद्वांसो यथा कुर्वन्ति भारत।\n\...,"ପୂର୍ବରୁ, ଶ୍ଲୋକ ସଂଖ୍ୟା ୩.୨୦ ରେ, ଶ୍ରୀକୃଷ୍ଣ ଲୋକ-ସ..."
480,12,19,तुल्यनिन्दास्तुतिर्मौनी सन्तुष्टो येनकेनचित्।अ...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ଭକ୍ତମାନଙ୍କର ଅନ୍ୟ ଦଶଟି ଗୁଣ ବର୍ଣ...
479,12,18,समः शत्रौ च मित्रे च तथा मानापमानयोः।शीतोष्णसु...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ଭକ୍ତମାନଙ୍କର ଅନ୍ୟ ଦଶଟି ଗୁଣ ବର୍ଣ...
222,5,26,कामक्रोधवियुक्तानां यतीनां यतचेतसाम्।\n\nअभितो...,ଶ୍ଲୋକ ୫.୨ରେ ବର୍ଣ୍ଣନା କରାଯାଇଛି ଯେ କର୍ମଯୋଗ ଅଧିକା...
65,2,26,अथ चैनं नित्यजातं नित्यं वा मन्यसे मृतम्।\n\nत...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ଅଥ ଶବ୍ଦ ପ୍ରୟୋଗ କରି ସୂଚିତ କରୁଛନ...


## Euclidean Distance

In [29]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [30]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
484,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,ବର୍ତ୍ତମାନ ଶ୍ରୀକୃଷ୍ଣ ନିଜେ ଅନେକ ପ୍ରଶ୍ନ ଉପସ୍ଥାପନ ...
83,2,44,भोगैश्वर्यप्रसक्तानां तयापहृतचेतसाम्।\n\nव्यवस...,"ଯେଉଁମାନଙ୍କର ମନ ଇନ୍ଦ୍ରିୟ ସୁଖରେ ଆସକ୍ତ ରହିଥାଏ, ସେ..."
512,13,32,अनादित्वान्निर्गुणत्वात्परमात्मायमव्ययः।शरीरस्...,"ଭଗବାନ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ହୃଦୟରେ, ପରମାତ୍ମା ରୂପରେ ନ..."
60,2,21,वेदाविनाशिनं नित्यं य एनमजमव्ययम्।\n\nकथं स पु...,"ଆଧ୍ୟାତ୍ମିକତାରେ ଯେଉଁମାନେ ଉଚ୍ଚ କକ୍ଷରେ ଅବସ୍ଥାପିତ,..."
76,2,37,हतो वा प्राप्स्यसि स्वर्गं जित्वा वा भोक्ष्यसे...,୨.୩୧ ଶ୍ଳୋକ ଠାରୁ ଆରମ୍ଭ କରି ଶ୍ରୀକୃଷ୍ଣ ଏ ପର୍ଯ୍ୟନ୍...
636,18,22,यत्तु कृत्स्नवदेकस्मिन्कार्ये सक्तमहैतुकम्।अतत...,"ତମୋଗୁଣ ପ୍ରଭାବରେ ବୁଦ୍ଧି ଯେତେବେଳେ ଶିଥିଳ ହୋଇଯାଏ, ..."
481,12,20,ये तु धर्म्यामृतमिदं यथोक्तं पर्युपासते।श्रद्द...,ଅର୍ଜୁନଙ୍କ ପ୍ରଶ୍ନର ଉତ୍ତର ଭାବରେ ଶ୍ରୀକୃଷ୍ଣ ଯାହା ଯ...
62,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"ଚୈତନ୍ୟତା, ଯାହା ଆତ୍ମାର ଲକ୍ଷଣ ଅଟେ, ତାହାର ଅନୁଭବ ମ..."
531,14,16,कर्मणः सुकृतस्याहुः सात्त्विकं निर्मलं फलम्।रज...,"ସତ୍ତ୍ୱଗୁଣ ଦ୍ୱାରା ପ୍ରଭାବିତ ବ୍ୟକ୍ତି ଶୁଦ୍ଧତା, ସଦ୍..."
672,18,58,मच्चित्तः सर्वदुर्गाणि मत्प्रसादात्तरिष्यसि।अथ...,"ପୂର୍ବ ଶ୍ଳୋକରେ କ’ଣ କରଣୀୟ ତାହାର ଉପଦେଶ ଦେବା ପରେ, ..."


# Whole Part

## Cosine Similarity

In [31]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [32]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
155,4,1,श्री भगवानुवाच\n\nइमं विवस्वते योगं प्रोक्तवान...,କାହାକୁ କେବଳ ଅମୂଲ୍ୟ ଜ୍ଞାନ ପ୍ରଦାନ କରିଦେଲେ ଯଥେଷ୍ଟ...
146,3,35,श्रेयान्स्वधर्मो विगुणः परधर्मात्स्वनुष्ठितात्...,ଏହି ଶ୍ଲୋକରେ ଧର୍ମ ଶବ୍ଦର ବ୍ୟବହାର ଚାରିଥର କରାଯାଇଛି...
283,7,11,बलं बलवतां चाहं कामरागविवर्जितम्।\n\nधर्माविरु...,ଅପ୍ରାପ୍ତ ବସ୍ତୁର ସକ୍ରିୟ କାମନାକୁ ବାସନା କୁହାଯାଏ ।...
249,6,24,सङ्कल्पप्रभवान्कामांस्त्यक्त्वा सर्वानशेषतः।\n...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
250,6,25,शनैः शनैरुपरमेद् बुद्ध्या धृतिगृहीतया।\n\nआत्म...,ଧ୍ୟାନ ପାଇଁ ଦୁଇଟି ପ୍ରକ୍ରିୟା ଆବଶ୍ୟକ: ସଂସାରରୁ ମନକ...
628,18,14,अधिष्ठानं तथा कर्ता करणं च पृथग्विधम्।विविधाश्...,"ଏହି ଶ୍ଳୋକରେ, ଅଧିଷ୍ଠାନଂ ଶବ୍ଦର ଅର୍ଥ “ବାସସ୍ଥାନ” ଅ..."
369,10,5,अहिंसा समता तुष्टिस्तपो दानं यशोऽयशः।\n\nभवन्त...,"ଏହି ଦୁଇଟି ଶ୍ଲୋକରେ, ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କର ପୂର୍ଣ୍ଣ ପ୍..."
368,10,4,बुद्धिर्ज्ञानमसंमोहः क्षमा सत्यं दमः शमः।\n\nस...,"ଏହି ଦୁଇଟି ଶ୍ଲୋକରେ, ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କର ପୂର୍ଣ୍ଣ ପ୍..."
78,2,39,एषा तेऽभिहिता सांख्ये बुद्धिर्योगे त्विमां श्र...,ସାଂଖ୍ୟ ଶବ୍ଦଟି ସାଂ ଅର୍ଥାତ୍ ‘ପୂର୍ଣ୍ଣ’ ଓ ଖ୍ୟ ଅର୍ଥ...


## Euclidean Distance

In [33]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [34]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
30,1,35,एतान्न हन्तुमिच्छामि घ्नतोऽपि मधुसूदन।\n\nअपि ...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
29,1,34,आचार्याः पितरः पुत्रास्तथैव च पितामहाः।\n\nमात...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
446,11,40,नमः पुरस्तादथ पृष्ठतस्ते\n\nनमोऽस्तु ते सर्वत ...,ଅର୍ଜୁନ ଶ୍ରୀକୃଷ୍ଣଙ୍କର ଐଶ୍ୱର୍ଯ୍ୟଗାନ କରିବାର କ୍ରମ ...
19,1,22,यावदेतान्निरीक्षेऽहं योद्धुकामानवस्थितान्।\n\n...,"ଅର୍ଜୁନ, ସମଗ୍ର ସୃଷ୍ଟିର ପରମ ପ୍ରଭୁ, ଶ୍ରୀକୃଷ୍ଣଙ୍କର..."
18,1,21,अर्जुन उवाच\n\nहृषीकेशं तदा वाक्यमिदमाह महीपते...,"ଅର୍ଜୁନ, ସମଗ୍ର ସୃଷ୍ଟିର ପରମ ପ୍ରଭୁ, ଶ୍ରୀକୃଷ୍ଣଙ୍କର..."
449,11,43,पितासि लोकस्य चराचरस्य\n\nत्वमस्य पूज्यश्च गुर...,"ଅର୍ଜୁନ କହୁଛନ୍ତି ଯେ, ଶ୍ରୀକୃଷ୍ଣ ସର୍ବଶ୍ରେଷ୍ଠ ତଥା ..."
401,10,37,वृष्णीनां वासुदेवोऽस्मि पाण्डवानां धनंजयः।\n\n...,ବୃଷ୍ଣୀବଂଶରେ ବସୁଦେବଙ୍କ ପୁତ୍ର ରୂପରେ ଶ୍ରୀକୃଷ୍ଣ ପୃ...
689,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम्।योगं य...,"ଶ୍ରୀକୃଷ୍ଣ ଦ୍ୱୈପାୟନ ବ୍ୟାସଦେବ, ଯିଏ ମହର୍ଷି ବେଦବ୍ୟ..."
675,18,61,ईश्वरः सर्वभूतानां हृद्देशेऽर्जुन तिष्ठति।भ्रा...,ଆତ୍ମାର ପରମାତ୍ମାଙ୍କ ଉପରେ ନିର୍ଭରଶୀଳତା ଉପରେ ଗୁରୁତ...
412,11,6,पश्यादित्यान्वसून्रुद्रानश्िवनौ मरुतस्तथा।\n\n...,ଭଗବାନଙ୍କର ବିଶ୍ୱରୂପରେ ପୃଥିବୀ ପୃଷ୍ଠର ସମସ୍ତ ଐଶ୍ଚର...


# Clustering

## Max Pooling

In [35]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [36]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [37]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
88,2,49,दूरेण ह्यवरं कर्म बुद्धियोगाद्धनञ्जय।\n\nबुद्ध...,କର୍ମର ଦୁଇଟି ଦିଗ ରହିଛି । ୧) ଆମେ କରୁଥିବା ବାହ୍ୟ କ...
122,3,11,देवान्भावयतानेन ते देवा भावयन्तु वः।\n\nपरस्पर...,ସ୍ୱର୍ଗର ଦେବତା ଗଣ ବିଶ୍ୱ ପରିଚାଳନାର ଦାୟିତ୍ୱ ନିର୍ବ...
155,4,1,श्री भगवानुवाच\n\nइमं विवस्वते योगं प्रोक्तवान...,କାହାକୁ କେବଳ ଅମୂଲ୍ୟ ଜ୍ଞାନ ପ୍ରଦାନ କରିଦେଲେ ଯଥେଷ୍ଟ...
217,5,21,बाह्यस्पर्शेष्वसक्तात्मा विन्दत्यात्मनि यत्सुख...,ବୈଦିକ ଶାସ୍ତ୍ରସବୁ ଭଗବାନଙ୍କୁ ଅନନ୍ତ ଦିବ୍ୟ ଆନନ୍ଦର ...
229,6,4,यदा हि नेन्द्रियार्थेषु न कर्मस्वनुषज्जते।\n\n...,ମନ ଯେତେବେଳେ ଯୋଗରେ ଭଗବାନଙ୍କ ସହିତ ସଂଯୁକ୍ତ ହୋଇଯାଏ...
233,6,8,ज्ञानविज्ञानतृप्तात्मा कूटस्थो विजितेन्द्रियः।...,"ଜ୍ଞାନର ତାତ୍ପର୍ଯ୍ୟ ଶାବ୍ଦିକ ଜ୍ଞାନ ଅଟେ, ଯାହା ଆମେ ..."
244,6,19,यथा दीपो निवातस्थो नेङ्गते सोपमा स्मृता।\n\nयो...,ଏହି ଶ୍ଲୋକରେ ଶ୍ରୀକୃଷ୍ଣ ଦୀପଶିଖାର ଉପମା ଦେଇଛନ୍ତି ।...
246,6,21,सुखमात्यन्तिकं यत्तद्बुद्धिग्राह्यमतीन्द्रियम्...,"ଆନନ୍ଦର କାମନା କରିବା, ଜୀବର ସହଜାତ ସ୍ୱଭାବ ଅଟେ । ଏହ..."
257,6,32,आत्मौपम्येन सर्वत्र समं पश्यति योऽर्जुन।\n\nसु...,ଆମେ ଶରୀରର ସମସ୍ତ ଅଙ୍ଗ ପ୍ରତ୍ୟଙ୍ଗକୁ ନିଜର ବୋଲି ବିଚ...
259,6,34,चञ्चलं हि मनः कृष्ण प्रमाथि बलवद्दृढम्।\n\nतस्...,"ଅର୍ଜୁନ, ଆମ ସମସ୍ତଙ୍କୁ ଲକ୍ଷ୍ୟ କରି, ଏହି ଉପଦ୍ରବୀ ମ..."


## Mean Pooling

In [38]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [39]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [40]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
62,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"ଚୈତନ୍ୟତା, ଯାହା ଆତ୍ମାର ଲକ୍ଷଣ ଅଟେ, ତାହାର ଅନୁଭବ ମ..."


## Whole

In [41]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [42]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [43]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
63,2,24,अच्छेद्योऽयमदाह्योऽयमक्लेद्योऽशोष्य एव च।\n\nन...,ଆତ୍ମାର ଅମରତ୍ୱ ବିଷୟରେ ଏଠାରେ ପୁନରାବୃତ୍ତି କରାଯାଇଛ...
73,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,ସମ୍ଭ୍ରାନ୍ତ ଶ୍ରେଣୀର ବ୍ୟକ୍ତିଙ୍କ ପାଇଁ ସାମାଜିକ ପ୍ର...
155,4,1,श्री भगवानुवाच\n\nइमं विवस्वते योगं प्रोक्तवान...,କାହାକୁ କେବଳ ଅମୂଲ୍ୟ ଜ୍ଞାନ ପ୍ରଦାନ କରିଦେଲେ ଯଥେଷ୍ଟ...
182,4,28,द्रव्ययज्ञास्तपोयज्ञा योगयज्ञास्तथापरे।\n\nस्व...,"ପ୍ରତ୍ୟେକ ମନୁଷ୍ୟ ତାଙ୍କର ସ୍ୱଭାବ, ଇଚ୍ଛା, କର୍ମ, ବୃ..."
186,4,32,एवं बहुविधा यज्ञा वितता ब्रह्मणो मुखे।\n\nकर्म...,"ବେଦର ଏକ ସୁନ୍ଦର ବୈଶିଷ୍ଟ୍ୟ ହେଲା, ଏହା ମାନବ ପ୍ରକୃତ..."
292,7,20,कामैस्तैस्तैर्हृतज्ञानाः प्रपद्यन्तेऽन्यदेवताः...,ପରମ ପୁରୁଷ ଶ୍ରୀକୃଷ୍ଣ ସୃଷ୍ଟିର ସବୁକିଛିର ଆଧାର ଅଟନ୍...
340,9,10,मयाऽध्यक्षेण प्रकृतिः सूयते सचराचरम्।\n\nहेतुन...,"ପୂର୍ବ ଶ୍ଳୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିବା ପ୍ରକାରେ, ଭଗବ..."
505,13,25,ध्यानेनात्मनि पश्यन्ति केचिदात्मानमात्मना।अन्य...,ଭଗବାନଙ୍କ ସୃଷ୍ଟିରେ ବିବିଧତା ଏକ ସାର୍ବଜନୀନ ବିଶେଷତ୍...
516,14,1,श्री भगवानुवाचपरं भूयः प्रवक्ष्यामि ज्ञानानां ...,"ପୂର୍ବ ଅଧ୍ୟାୟରେ, ଶ୍ରୀକୃଷ୍ଣ ବର୍ଣ୍ଣନା କରିଥିଲେ ଯେ ..."
604,17,18,सत्कारमानपूजार्थं तपो दम्भेन चैव यत्।क्रियते त...,ଆତ୍ମଶୁଦ୍ଧି ପାଇଁ ତପସ୍ୟା ଏକ ଶକ୍ତିଶାଳୀ ମାଧ୍ୟମ ହୋଇ...
